In [1]:
import random
import numpy as np

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import os
import cv2
import urllib.request
import matplotlib.pyplot as plt
from IPython.display import clear_output

### Principal functions

In [ ]:
# load image from url
def load_target_image(image_url, color=True, size=None):
  image_path = "target_image"    
  urllib.request.urlretrieve(image_url,image_path)
  # Apply color
  if color:
      target = cv2.imread(image_path, cv2.IMREAD_COLOR)
      target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)
  else:
      # Apply gray scale
      target = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

  if size:
      # Only resizes image if it is needed!
      target = cv2.resize(src=target, dsize=size, interpolation=cv2.INTER_AREA)
  return target

In [2]:
# Helps to show image
def show_image(img_arr):    
  plt.figure(figsize=(10,10))
  plt.axis("off")
  plt.imshow(img_arr/255)
  plt.show()

In [3]:
# Show results
def show_results(history, img_arr, org):
  plt.figure(figsize=(10,10))
  plt.tight_layout()

  plt.subplot(221)
  plt.axis("off")
  plt.imshow(img_arr/255)
  plt.title('best of generation')

  plt.subplot(222)
  plt.axis("off")
  plt.imshow(org/255)
  plt.title('target image')

  plt.subplot(212)
  lh = len(history)
  plt.xlim([lh-50, lh])
  plt.plot(history)
  plt.title('min fitness by generation') 
  plt.show()